<b> 24 folders are Uploded into G drive for each actor (contains 1440 files: 60 trials per actor x 24 actors = 1440, total files). First we mount G drive into the colab for accessing the .wav files<b>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


<b> The Encoding of the files are as follows <a href="https://zenodo.org/record/1188976#.XSF23-hKiM9">RAVDESS</a></b>

   <li>Modality (01 = full-AV, 02 = video-only, 03 = audio-only).</li>
    <li>Vocal channel (01 = speech, 02 = song).</li>
    <li>Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).</li>
    <li>Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the ‘neutral’ emotion.</li>
    <li>Statement (01 = “Kids are talking by the door”, 02 = “Dogs are sitting by the door”).</li>
    <li>Repetition (01 = 1st repetition, 02 = 2nd repetition).</li>
    <li>Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).</li>



<b>Filename example: 02-01-06-01-02-01-12.mp4</b>

<li>Video-only (02)
Speech (01)
Fearful (06)
Normal intensity (01)
Statement "dogs" (02)
1st Repetition (01)
12th Actor (12)
Female, as the actor ID number is even.</li>


<b> Some common Sound related terms</b>

<b>Audio Frame</b>
<li>An audio frame, or sample, contains amplitude (loudness) information at that particular point in time. To produce sound, tens of thousands of frames are played in sequence to produce frequencies.</li>
<b>Bit Rate</b>

<li> refers to the audio quality of the stream. It is measured in Kilobitspersec(kbps or k). Bit rate is no of bits (data) encoded per second or the no. of bits transmitted or received per second. Higher the bit rate with more sampling rate, requires high bandwidth and produces good audio quality.<li>
  
  
  
<b>Sampling Rate </b>
<li>Sampling rate (sometimes called sampling frequency or Fs) is the number of data points acquired per second. 


 

A sampling rate of 2000 samples/second means that 2000 discrete data points are acquired every second.  This can be referred to as 2000 Hertz sample frequency</li>


In [0]:
### set Main directory for all the data
Main_dir = '/content/gdrive/My Drive/sounds/Actor_'

In [0]:
### Import all Dependencies 
import time
import matplotlib.pyplot as plt
%matplotlib inline
import librosa as lr
from glob import glob
import pandas as pd
import numpy as np
import IPython.display as ipd
#audio,sfreq=lr.load(path,sr=None)
import librosa.display

<li>I read all the files from the directory using librosa to calculate sampling rate and audiot time series  and calculate duration of each for each wav file and store in a dictionary file_data, with file path as key and value as length of file. The second dictionary sampling_rate_file  has keys as path of files and value as sampling rate </li>

In [0]:
### loop over 24 files, check length of file if length of file is 1 then add 0
### to the prefix and add to the main directory for globing files for actors 01-09,
### if lenght of files greater than 1 then add add to main directory for globing files for actor 10-24
file_data,sampling_rate_file = {},{}
for files in range(1,25):
    if len(str(files))==1:
        data_dir = Main_dir
        data_dir = data_dir+str('0')+str(files)
        #print(data_dir)
    else:
        data_dir = Main_dir
        data_dir = data_dir+str(files)
        #print(data_dir)
    audio_files = glob(data_dir+'/*.wav')
    for i in range(len(audio_files)):
      x= audio_files[i]
      audio,sfreq=lr.load(audio_files[i],sr=None)
      file_data[x]= len(audio)/sfreq
      sampling_rate_file[x]=sfreq

<li>check for some values of the dictinary for values of sampling_rate_file dictionary to view sample of sampling rates </li>


In [8]:
[sampling_rate_file[i] for i in list(sampling_rate_file.keys())[:10]]

[48000, 48000, 48000, 48000, 48000, 48000, 48000, 48000, 48000, 48000]

<li> check if all values in the dictionary are 48000 or 48khz sampling rate </li>

In [10]:
all(value == 48000 for value in sampling_rate_file.values())

True

In [0]:
## Create a Data Frame with file path as indexes and lengthofile column as lenght of each file in seconds

data_df=pd.DataFrame()
for keys,values in file_data.items():
    data_df.at[keys,'lengthoffile']=values

In [14]:
### preview Data Frame
print('shape of the data frame is {data_df.shape}')
print('preview of data Frame')
data_df.head(10)

shape of the data frame is {data_df.shape}
preview of data Frame


,lengthoffile
/content/gdrive/My Drive/sounds/Actor_01/03-01-01-01-01-01-01.wav,3.303292
/content/gdrive/My Drive/sounds/Actor_01/03-01-01-01-01-02-01.wav,3.336667
/content/gdrive/My Drive/sounds/Actor_01/03-01-01-01-02-01-01.wav,3.269917
/content/gdrive/My Drive/sounds/Actor_01/03-01-02-01-01-01-01.wav,3.536854
/content/gdrive/My Drive/sounds/Actor_01/03-01-02-01-01-02-01.wav,3.603583
/content/gdrive/My Drive/sounds/Actor_01/03-01-01-01-02-02-01.wav,3.169833
/content/gdrive/My Drive/sounds/Actor_01/03-01-02-01-02-01-01.wav,3.503500
/content/gdrive/My Drive/sounds/Actor_01/03-01-03-01-01-02-01.wav,3.470125
/content/gdrive/My Drive/sounds/Actor_01/03-01-02-02-01-01-01.wav,3.703708
/content/gdrive/My Drive/sounds/Actor_01/03-01-02-01-02-02-01.wav,3.470146


###Explore data for  one sample File
<li> by rule of Encoding odd number are male Male_file = data_df.index[0]</li>

As per documentation ( <a href= "https://librosa.github.io/librosa/generated/librosa.core.load.html"> librosa</a>)
load function retruns 


<li>y:np.ndarray [shape=(n,) or (2, n)],audio time series</li>

<li>sr:number > 0 [scalar] sampling rate of y</li>

In [18]:
audio,sfreq=audio_data(data_df.index[0])
print(f'shape of audio time series {audio.shape} sampling rate {sfreq}')

shape of audio time series (158558,) sampling rate 48000


### Goal: Using the Audio time series returned by librosa.load function we Extract mfcc for each file and use file name encoding to extract labels for data

<b> mfcc </b>

The mel-frequency cepstral coefficients are computed over a window, i.e. number of samples. Sound is wave and one cannot derive any features by taking a single sample (number), hence the window.

To compute MFCC, fast Fourier transform (FFT) is used and that exactly requires that length of a window is provided. In librosa , the below are implicit specifically:  
<li>length of the FFT window: 2048</li>
<li>number of samples between successive frames: 512</li>

If a cepstral coefficient has a positive value, it represents a sonorant sound since the majority of the spectral energy in sonorant sounds are concentrated in the low-frequency regions.
- On the other hand, if a cepstral coefficient has a negative value, it represents a fricative sound since most of the spectral energies in fricative sounds are concentrated at high frequencies.
- The lower order coefficients contain most of the information about the overall spectral shape of the source-filter transfer function.
- The zero-order coefficient indicates the average power of the input signal.
- The first-order coefficient represents the distribution spectral energy between low and high frequencies.


In [0]:
### Create a Function to return the Aduio Data samples and Sampling rate for each file
def audio_data(file):
  audio,sfreq = lr.load(file,sr=None)
  return audio,sfreq



### Create a function to extract Mfcc for a given file
def mfcc_extract(file):
  audio,sfreq = audio_data(file)
  mfccs = librosa.feature.mfcc(audio, sr=sfreq)
  return mfccs


<b> Extract a mfcc from one sample file and explore the data</b>

In [26]:

print(f'the shape of sample file  mfccs is {mfcc_extract(data_df.index[0]).shape}')

the shape of sample file  mfccs is (20, 310)


<li> The Shape for the male sample file gives us 20 coefficents/features  and 310 samples. using sampling rate as 48000 and audio time duration as 3.3 seconds ( rounded), we can calulcate nfcc = len*sampling/window = (3.3*48000)/512 ~310 i.e the shape we get from librosa.feature.mfcc, similarily we can arrive at the shape returned for the female file as we can see they are not balanced arrays</li>


### Create a data frame with all mfcc data from 1440 files and Extract labels from the encoding as descibed above

<li>Create a list of data frames from mfcc, add a columns for coding of actor files
concat the data frames for final data frame which hold all the data from 1440 filesl</li>

In [0]:
### Empty list
list_data_frame=[]
### Loop over each file path in Data_df dataframe
for file_path in data_df.index:
  data=mfcc_extract(file_path)### call mfcc extract to return mfcc coeffcients and data
  frame=pd.DataFrame(data.T,columns=['mfcc'+str(x) for x in range(0,20)])#### Transform data into (nsamples,n features)
  frame['Actor_Gender_code'] = file_path[38:40]### Extract Actor Gender from file name using defined Encoding
  frame['Emotion_code']=file_path[47:49] #### Extract Emotion code from file name using defined encoding
  frame['Emotional_intensity_code']=file_path[50:52] #### Extract Emotional Intensity code using identified encoding
  list_data_frame.append(frame)
  
  
  
### Final Data Frame
merged_df = pd.concat(list_data_frame,ignore_index=True) 
  

<li> Preview the Data </li>


In [36]:
print(f'shape of the final data frame {merged_df.shape}, we see the number of samples over 20 features/MFCC is {merged_df.shape[0]}')
merged_df.head()


shape of the final data frame (500286, 23), we see the number of samples over 20 features/MFCC is 500286


,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,Actor_Gender_code,Emotion_code,Emotional_intensity_code
0,-861.532578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,01,01
1,-861.532578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,01,01
2,-861.532578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,01,01
3,-861.532578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,01,01
4,-861.532578,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,01,01,01


<b>Create labels for Gender,emotion and emotion intensity from the code derived </b>

In [0]:
### Create Male female from gender code , odd is male and even is female
merged_df["Gender_label"] = list(map(lambda x: 'Male' if int(x)%2==1 else 'Female',merged_df.Actor_Gender_code))

### Emotion label for various classes 
merged_df['Emotion_label']=merged_df.Emotion_code.map({'01':'neutral','02':'calm','03':'happy','04':'sad','05':'angry','06':'fearful','07':'disgust','08':'surprised'})
### Emotional Intensity normal or strong
merged_df['Intensity_label']=merged_df.Emotional_intensity_code.map({'01':'normal','02':'strong'})

<li><b>Preview data </b></li>

In [41]:

merged_df.tail(10)

,mfcc0,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,Actor_Gender_code,Emotion_code,Emotional_intensity_code,Gender_label,Emotion_label,Intensity_label
500276,-689.920585,8.210541,8.121399,7.978220,7.788671,7.562674,7.311673,7.047788,6.782929,6.527924,6.291747,6.080888,5.898917,5.746284,5.620354,5.515684,5.424521,5.337472,5.244302,5.134797,24,08,02,Female,surprised,strong
500277,-691.880154,5.463201,5.444165,5.412559,5.368562,5.312420,5.244452,5.165036,5.074617,4.973695,4.862827,4.742618,4.613721,4.476828,4.332666,4.181992,4.025587,3.864251,3.698794,3.530033,24,08,02,Female,surprised,strong
500278,-691.858668,5.490103,5.460660,5.411840,5.344022,5.257730,5.153626,5.032511,4.895309,4.743064,4.576927,4.398148,4.208059,4.008068,3.799639,3.584285,3.363549,3.138990,2.912174,2.684652,24,08,02,Female,surprised,strong
500279,-691.228681,6.379412,6.345112,6.288286,6.209442,6.109281,5.988692,5.848737,5.690643,5.515785,5.325669,5.121918,4.906248,4.680453,4.446383,4.205921,3.960964,3.713404,3.465103,3.217877,24,08,02,Female,surprised,strong
500280,-692.569667,4.481327,4.442905,4.381834,4.302277,4.209530,4.109539,4.008371,3.911672,3.824170,3.749271,3.688774,3.642742,3.609540,3.586025,3.567888,3.550092,3.527388,3.494843,3.448341,24,08,02,Female,surprised,strong
500281,-693.609022,3.021236,3.011290,2.994892,2.972303,2.943883,2.910079,2.871417,2.828491,2.781950,2.732479,2.680791,2.627607,2.573642,2.519586,2.466096,2.413777,2.363167,2.314735,2.268863,24,08,02,Female,surprised,strong
500282,-691.884404,5.430296,5.332270,5.174513,4.964974,4.713867,4.432866,4.134216,3.829841,3.530532,3.245297,2.980922,2.741782,2.529897,2.345226,2.186138,2.050018,1.933900,1.835086,1.751647,24,08,02,Female,surprised,strong
500283,-692.399969,4.718255,4.670174,4.591483,4.484294,4.351463,4.196495,4.023424,3.836675,3.640924,3.440934,3.241409,3.046836,2.861345,2.688584,2.531609,2.392798,2.273796,2.175475,2.097935,24,08,02,Female,surprised,strong
500284,-694.268350,2.090835,2.086927,2.080427,2.071357,2.059745,2.045628,2.029049,2.010063,1.988728,1.965110,1.939285,1.911330,1.881334,1.849386,1.815585,1.780033,1.742835,1.704102,1.663949,24,08,02,Female,surprised,strong
500285,-692.471778,4.618038,4.574524,4.505245,4.414798,4.309120,4.195019,4.079656,3.969989,3.872242,3.791426,3.730959,3.692409,3.675385,3.677578,3.694948,3.722058,3.752502,3.779415,3.796019,24,08,02,Female,surprised,strong
